# Setup

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%%capture
!pip install datasets evaluate transformers
!pip install rouge_score
!pip install accelerate

In [6]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, TrainingArguments, Seq2SeqTrainingArguments
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
import pandas as pd
from datasets import load_metric
import torch
import numpy as np

In [7]:
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base")
# model.to('cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

# Preprocessing data

In [1]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inputs"], max_length=300, truncation=True, padding=True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["labels"], max_length=300, truncation=True, padding=True
        )
    model_inputs['labels'] = labels['input_ids']
    model_inputs['input_ids'] = model_inputs['input_ids']
    return model_inputs

In [2]:
train_set = pd.read_csv("/content/drive/MyDrive/CT208/Data/train_set.csv").astype("str")
val_set = pd.read_csv("/content/drive/MyDrive/CT208/Data/val_set.csv")
test_set = pd.read_csv("/content/drive/MyDrive/CT208/Data/test_set.csv")

NameError: name 'pd' is not defined

In [ ]:
def tokenize(data):
    input_lines = data.iloc[:, 0].to_numpy()
    label_lines = data.iloc[:, 1].to_numpy()
    dict_obj = {'inputs': input_lines, 'labels': label_lines}
    dataset = Dataset.from_dict(dict_obj)
    tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=10)

    return tokenized_datasets

In [ ]:
tokenized_train_set = tokenize(train_set)
tokenized_val_set = tokenize(val_set)
tokenized_test_set = tokenize(test_set)

In [ ]:
print(len(tokenized_train_set))
print(len(tokenized_val_set))
print(len(tokenized_test_set))

# Draft

## Training

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

training_args = Seq2SeqTrainingArguments("/content/drive/MyDrive/CT208/1st",
                                      do_train=True,
                                      do_eval=False,
                                      num_train_epochs=30,
                                      learning_rate=1e-5,
                                      warmup_ratio=0.05,
                                      weight_decay=0.01,
                                      per_device_train_batch_size=4,
                                      per_device_eval_batch_size=4,
                                      logging_dir='./log',
                                      group_by_length=True,
                                      save_strategy="epoch",
                                      save_total_limit=3,
                                      #eval_steps=1,
                                      #evaluation_strategy="steps",
                                      # evaluation_strategy="no",
                                      fp16=True,
                                      )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_set,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

## Evaluate

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/CT208/1st/checkpoint-22500")
model.to('cuda')

In [ ]:
metrics = load_metric('rouge')

max_target_length = 300
dataloader = torch.utils.data.DataLoader(tokenized_val_set, collate_fn=data_collator, batch_size=32)

predictions = []
references = []
for i, batch in enumerate(tqdm(dataloader)):
    outputs = model.generate(
        input_ids=batch['input_ids'].to('cuda'),
        max_length=max_target_length,
        attention_mask=batch['attention_mask'].to('cuda'),
    )
    with tokenizer.as_target_tokenizer():
        outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

        labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
        actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
    predictions.extend(outputs)
    references.extend(actuals)
    metrics.add_batch(predictions=outputs, references=actuals)


metrics.compute()

In [ ]:
[{k: v.mid.fmeasure} for k,v in metrics.compute(predictions=predictions, references=references).items()]

## Generate text

In [ ]:
sentence = "Các giá trị 'rouge-1,' 'rouge-2,' 'rouge-L,' và 'rouge-Lsum' là các phương pháp đánh giá chất lượng của các hệ thống tạo ra các văn bản tóm tắt. Đây là các độ đo phổ biến trong lĩnh vực xử lý ngôn ngữ tự nhiên và tóm tắt máy học."
text =  sentence + " </s>"
encoding = tokenizer(text, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")
outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=300,
    early_stopping=True
)
for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(line)

# 1st

## Training

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

training_args = Seq2SeqTrainingArguments(output_dir="tmp/",
                                      do_train=True,
                                      do_eval=True,
                                      num_train_epochs=30,
                                      learning_rate=1e-5,
                                      warmup_ratio=0.05,
                                      weight_decay=0.01,
                                      per_device_train_batch_size=4,
                                      per_device_eval_batch_size=4,
                                      logging_dir='./log',
                                      group_by_length=True,
                                      save_strategy="epoch",
                                      save_total_limit=1,
                                      evaluation_strategy="epoch",
                                      fp16=True,
                                      )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_set,
    data_collator=data_collator,
    eval_dataset=tokenized_val_set
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.003900,0.088920
2,0.004000,0.093458
3,0.003800,0.091270
4,0.003100,0.094107
5,0.002300,0.095836
6,0.002100,0.097693
7,0.002400,0.094863
8,0.001800,0.094528
9,0.001700,0.094873
10,0.001800,0.096007


KeyboardInterrupt: 

## Evaluate

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("/content/tmp/checkpoint-15000")
model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
metrics = load_metric('rouge')

max_target_length = 300
dataloader = torch.utils.data.DataLoader(tokenized_test_set, collate_fn=data_collator, batch_size=32)

predictions = []
references = []
for i, batch in enumerate(tqdm(dataloader)):
    outputs = model.generate(
        input_ids=batch['input_ids'].to('cuda'),
        max_length=max_target_length,
        attention_mask=batch['attention_mask'].to('cuda'),
    )
    with tokenizer.as_target_tokenizer():
        outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

        labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
        actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
    predictions.extend(outputs)
    references.extend(actuals)
    metrics.add_batch(predictions=outputs, references=actuals)


metrics.compute()

<ipython-input-22-4a8b4e0b284a>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metrics = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


  0%|          | 0/32 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'rouge1': AggregateScore(low=Score(precision=0.9697815817270669, recall=0.9649854988936158, fmeasure=0.9671411094961017), mid=Score(precision=0.9727936207335566, recall=0.968095967381642, fmeasure=0.9700114273291586), high=Score(precision=0.9756581854340024, recall=0.9711486167475941, fmeasure=0.9728402048995053)),
 'rouge2': AggregateScore(low=Score(precision=0.9475699001744455, recall=0.9428342865191611, fmeasure=0.9449278929941518), mid=Score(precision=0.9525957043810401, recall=0.9480341669488486, fmeasure=0.9499068165659356), high=Score(precision=0.9567887374436481, recall=0.9527548819803037, fmeasure=0.9542870401165016)),
 'rougeL': AggregateScore(low=Score(precision=0.9667994457184388, recall=0.9619037732888849, fmeasure=0.9640355081020906), mid=Score(precision=0.9701182577607471, recall=0.9655068392826804, fmeasure=0.9674139893836349), high=Score(precision=0.9732413702777974, recall=0.9687075037777059, fmeasure=0.9705334693640532)),
 'rougeLsum': AggregateScore(low=Score(preci

In [ ]:
[{k: v.mid.fmeasure} for k,v in metrics.compute(predictions=predictions, references=references).items()]

[{'rouge1': 0.9700114273291586},
 {'rouge2': 0.9499068165659356},
 {'rougeL': 0.9674139893836349},
 {'rougeLsum': 0.9674806461226331}]

## Generate text

In [ ]:
sentence = "Ngay 5 2 ban to chuc cuoc thi Thu thach Vesuvius da trao giai thuong tri gia 700 000 USD cho 3 nha nghien cuu ve thanh tuu ung dung tri tue nhan tao AI de giai ma thong tin trong cuon giay 2 000 nam tuoi bi chay sem trong vu phun trao nui lua Vesuvius chon vui thanh pho La Ma co dai Pompeii"
text =  sentence + " </s>"
encoding = tokenizer(text, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")
outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=300,
    early_stopping=True
)
for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(line)

Ngày 5 2 ban tổ chức cuộc thi Thu hoạch Vesuvius đã trao giải thưởng trị giá 700 000 USD cho 3 nhà nghiên cứu về thành tựu ứng dụng trí tuệ nhân tạo AI để giải mã thông tin trong cuốn sách 2 000 năm tuổi bị cháy sém trong vụ cháy rừng núi Vesuvius chọn vui thành phố La Mã cổ đại Pompeii 


NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

# 2nd

## Training

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

training_args = Seq2SeqTrainingArguments(output_dir="tmp/",
                                      do_train=True,
                                      do_eval=True,
                                      num_train_epochs=20,
                                      learning_rate=1e-5,
                                      warmup_ratio=0.05,
                                      weight_decay=0.01,
                                      per_device_train_batch_size=8,
                                      per_device_eval_batch_size=8,
                                      logging_dir='./log',
                                      group_by_length=True,
                                      save_strategy="epoch",
                                      save_total_limit=1,
                                      evaluation_strategy="epoch",
                                      fp16=True,
                                      )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_set,
    data_collator=data_collator,
    eval_dataset=tokenized_val_set
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.284200,0.140286
2,0.089200,0.049880
3,0.050200,0.031450
4,0.035900,0.024392
5,0.025200,0.020294
6,0.020200,0.018651
7,0.016500,0.017259
8,0.012900,0.016916
9,0.010500,0.016079
10,0.008900,0.015936


## Evaluate

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("/content/tmp/checkpoint-15000")
model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
metrics = load_metric('rouge')

max_target_length = 300
dataloader = torch.utils.data.DataLoader(tokenized_test_set, collate_fn=data_collator, batch_size=32)

predictions = []
references = []
for i, batch in enumerate(tqdm(dataloader)):
    outputs = model.generate(
        input_ids=batch['input_ids'].to('cuda'),
        max_length=max_target_length,
        attention_mask=batch['attention_mask'].to('cuda'),
    )
    with tokenizer.as_target_tokenizer():
        outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

        labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
        actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
    predictions.extend(outputs)
    references.extend(actuals)
    metrics.add_batch(predictions=outputs, references=actuals)


metrics.compute()

<ipython-input-22-4a8b4e0b284a>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metrics = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


  0%|          | 0/32 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'rouge1': AggregateScore(low=Score(precision=0.9697815817270669, recall=0.9649854988936158, fmeasure=0.9671411094961017), mid=Score(precision=0.9727936207335566, recall=0.968095967381642, fmeasure=0.9700114273291586), high=Score(precision=0.9756581854340024, recall=0.9711486167475941, fmeasure=0.9728402048995053)),
 'rouge2': AggregateScore(low=Score(precision=0.9475699001744455, recall=0.9428342865191611, fmeasure=0.9449278929941518), mid=Score(precision=0.9525957043810401, recall=0.9480341669488486, fmeasure=0.9499068165659356), high=Score(precision=0.9567887374436481, recall=0.9527548819803037, fmeasure=0.9542870401165016)),
 'rougeL': AggregateScore(low=Score(precision=0.9667994457184388, recall=0.9619037732888849, fmeasure=0.9640355081020906), mid=Score(precision=0.9701182577607471, recall=0.9655068392826804, fmeasure=0.9674139893836349), high=Score(precision=0.9732413702777974, recall=0.9687075037777059, fmeasure=0.9705334693640532)),
 'rougeLsum': AggregateScore(low=Score(preci

In [ ]:
[{k: v.mid.fmeasure} for k,v in metrics.compute(predictions=predictions, references=references).items()]

[{'rouge1': 0.9700114273291586},
 {'rouge2': 0.9499068165659356},
 {'rougeL': 0.9674139893836349},
 {'rougeLsum': 0.9674806461226331}]

## Generate text

In [ ]:
sentence = "Ngay 5 2 ban to chuc cuoc thi Thu thach Vesuvius da trao giai thuong tri gia 700 000 USD cho 3 nha nghien cuu ve thanh tuu ung dung tri tue nhan tao AI de giai ma thong tin trong cuon giay 2 000 nam tuoi bi chay sem trong vu phun trao nui lua Vesuvius chon vui thanh pho La Ma co dai Pompeii"
text =  sentence + " </s>"
encoding = tokenizer(text, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")
outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=300,
    early_stopping=True
)
for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(line)

Ngày 5 2 ban tổ chức cuộc thi Thu hoạch Vesuvius đã trao giải thưởng trị giá 700 000 USD cho 3 nhà nghiên cứu về thành tựu ứng dụng trí tuệ nhân tạo AI để giải mã thông tin trong cuốn sách 2 000 năm tuổi bị cháy sém trong vụ cháy rừng núi Vesuvius chọn vui thành phố La Mã cổ đại Pompeii 


NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

# 3rd

## Training

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

training_args = Seq2SeqTrainingArguments(
    output_dir="tmp/",
    do_train=True,
    do_eval=True,
    num_train_epochs=10,
    learning_rate=1e-5,
    warmup_ratio=0.05,
    weight_decay=0.01,
    prediction_loss_only = True,
    gradient_accumulation_steps = 3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./log',
    group_by_length=True,
    save_strategy="epoch",
    save_total_limit=1,
    evaluation_strategy="epoch",
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_set,
    data_collator=data_collator,
    eval_dataset=tokenized_val_set
)

In [ ]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
0,4.343900,0.178067
1,0.189600,0.080621
3,0.075700,0.042908
4,0.059900,0.036802
6,0.043500,0.030427
7,0.040600,0.028967
9,0.035800,0.027902


TrainOutput(global_step=9690, training_loss=0.3022702155590549, metrics={'train_runtime': 8856.819, 'train_samples_per_second': 26.276, 'train_steps_per_second': 1.094, 'total_flos': 5.06499773976576e+16, 'train_loss': 0.3022702155590549, 'epoch': 9.99})

In [ ]:
!mv /content/log/events.out.tfevents.1707365662.dbb41ac8b909.854.0 /content/drive/MyDrive/CT208/2rd

## Evaluate

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/CT208/2rd/checkpoint-9690")
model.to('cuda')

In [ ]:
metrics = load_metric('rouge')

max_target_length = 300
dataloader = torch.utils.data.DataLoader(tokenized_test_set, collate_fn=data_collator, batch_size=32)

predictions = []
references = []
for i, batch in enumerate(tqdm(dataloader)):
    outputs = model.generate(
        input_ids=batch['input_ids'].to('cuda'),
        max_length=max_target_length,
        attention_mask=batch['attention_mask'].to('cuda'),
    )
    with tokenizer.as_target_tokenizer():
        outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

        labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
        actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
    predictions.extend(outputs)
    references.extend(actuals)
    metrics.add_batch(predictions=outputs, references=actuals)

<ipython-input-17-4a8b4e0b284a>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metrics = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


  0%|          | 0/156 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'rouge1': AggregateScore(low=Score(precision=0.9834747456030071, recall=0.9819610463565717, fmeasure=0.9826422890804603), mid=Score(precision=0.984335839069195, recall=0.9828259745056115, fmeasure=0.9834553737075293), high=Score(precision=0.9851464778726258, recall=0.9836982404695586, fmeasure=0.984279538783495)),
 'rouge2': AggregateScore(low=Score(precision=0.9716482581189224, recall=0.9700250787394225, fmeasure=0.9707623506365148), mid=Score(precision=0.9730215106721225, recall=0.9715156641903945, fmeasure=0.9721647710199142), high=Score(precision=0.9743620065939519, recall=0.9729230313365992, fmeasure=0.9735553828918491)),
 'rougeL': AggregateScore(low=Score(precision=0.9824504386590022, recall=0.9808880918698047, fmeasure=0.981579063391171), mid=Score(precision=0.9834053482518403, recall=0.9818796565565557, fmeasure=0.9825250459119504), high=Score(precision=0.9843379333680138, recall=0.9827983958249998, fmeasure=0.9834433858242191)),
 'rougeLsum': AggregateScore(low=Score(precisi

In [ ]:
[{k: v.mid.fmeasure} for k,v in metrics.compute(predictions=predictions, references=references).items()]

[{'rouge1': 0.9834553737075293},
 {'rouge2': 0.9721647710199142},
 {'rougeL': 0.9825250459119504},
 {'rougeLsum': 0.9825770888165091}]

## Generate text

In [ ]:
sentence = input()
text =  sentence + " </s>"
encoding = tokenizer(text, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")
outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=300,
    early_stopping=True
)
for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(line)

lam sao de giai quyet van de nay day?
Làm sao để giải quyết vấn đề này đây? 


# 4th

## Training

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/CT208/4th",
    do_train=True,
    do_eval=True,
    num_train_epochs=6,
    learning_rate=1e-5,
    warmup_ratio=0.05,
    weight_decay=0.01,
    prediction_loss_only = True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_dir='/content/drive/MyDrive/CT208/4th',
    group_by_length=True,
    save_strategy="epoch",
    save_total_limit=1,
    evaluation_strategy="epoch",
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_set,
    data_collator=data_collator,
    eval_dataset=tokenized_val_set
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.078500,0.072845
2,0.041900,0.046855


## Evaluate

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/CT208/4th/checkpoint-23768")
model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
metrics = load_metric('rouge')

max_target_length = 300
dataloader = torch.utils.data.DataLoader(tokenized_test_set, collate_fn=data_collator, batch_size=32)

predictions = []
references = []
for i, batch in enumerate(tqdm(dataloader)):
    outputs = model.generate(
        input_ids=batch['input_ids'].to('cuda'),
        max_length=max_target_length,
        attention_mask=batch['attention_mask'].to('cuda'),
    )
    with tokenizer.as_target_tokenizer():
        outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

        labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
        actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
    predictions.extend(outputs)
    references.extend(actuals)
    metrics.add_batch(predictions=outputs, references=actuals)

<ipython-input-17-4a8b4e0b284a>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metrics = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


  0%|          | 0/156 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'rouge1': AggregateScore(low=Score(precision=0.9834747456030071, recall=0.9819610463565717, fmeasure=0.9826422890804603), mid=Score(precision=0.984335839069195, recall=0.9828259745056115, fmeasure=0.9834553737075293), high=Score(precision=0.9851464778726258, recall=0.9836982404695586, fmeasure=0.984279538783495)),
 'rouge2': AggregateScore(low=Score(precision=0.9716482581189224, recall=0.9700250787394225, fmeasure=0.9707623506365148), mid=Score(precision=0.9730215106721225, recall=0.9715156641903945, fmeasure=0.9721647710199142), high=Score(precision=0.9743620065939519, recall=0.9729230313365992, fmeasure=0.9735553828918491)),
 'rougeL': AggregateScore(low=Score(precision=0.9824504386590022, recall=0.9808880918698047, fmeasure=0.981579063391171), mid=Score(precision=0.9834053482518403, recall=0.9818796565565557, fmeasure=0.9825250459119504), high=Score(precision=0.9843379333680138, recall=0.9827983958249998, fmeasure=0.9834433858242191)),
 'rougeLsum': AggregateScore(low=Score(precisi

In [ ]:
[{k: v.mid.fmeasure} for k,v in metrics.compute(predictions=predictions, references=references).items()]

[{'rouge1': 0.9834553737075293},
 {'rouge2': 0.9721647710199142},
 {'rougeL': 0.9825250459119504},
 {'rougeLsum': 0.9825770888165091}]

## Generate text

In [ ]:
sentence = input()
text =  sentence + " </s>"
encoding = tokenizer(text, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")
outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=300,
    early_stopping=True
)
for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(line)

Canh sat cho biet Kiptum la nguoi dieu khien chiec xe mat lai va lan banh khien ca hai tu vong tai cho Mot phat ngon vien duoc AFP dan loi noi them rang hanh khach thu ba la nu bi thuong va duoc dua den benh vien
Cảnh sát cho biết Kiptum là người điều khiển chiếc xe mất lái và lăn bánh khiến cả hai tử vong tại chỗ Một phát ngôn viên được AFP dẫn lời nói thêm rằng hành khách thứ ba là nữ bị thương và được đưa đến bệnh viện 


# 5th

## Training

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/CT208/5th",
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    learning_rate=1e-5,
    warmup_ratio=0.05,
    weight_decay=0.01,
    prediction_loss_only = True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_dir='/content/drive/MyDrive/CT208/5th',
    group_by_length=True,
    save_strategy="epoch",
    save_total_limit=1,
    evaluation_strategy="epoch",
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_set,
    data_collator=data_collator,
    eval_dataset=tokenized_val_set
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.078500,0.072845
2,0.041900,0.046855


## Evaluate

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/CT208/4th/checkpoint-23768")
model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
metrics = load_metric('rouge')

max_target_length = 300
dataloader = torch.utils.data.DataLoader(tokenized_test_set, collate_fn=data_collator, batch_size=32)

predictions = []
references = []
for i, batch in enumerate(tqdm(dataloader)):
    outputs = model.generate(
        input_ids=batch['input_ids'].to('cuda'),
        max_length=max_target_length,
        attention_mask=batch['attention_mask'].to('cuda'),
    )
    with tokenizer.as_target_tokenizer():
        outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

        labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
        actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
    predictions.extend(outputs)
    references.extend(actuals)
    metrics.add_batch(predictions=outputs, references=actuals)

<ipython-input-17-4a8b4e0b284a>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metrics = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


  0%|          | 0/156 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'rouge1': AggregateScore(low=Score(precision=0.9834747456030071, recall=0.9819610463565717, fmeasure=0.9826422890804603), mid=Score(precision=0.984335839069195, recall=0.9828259745056115, fmeasure=0.9834553737075293), high=Score(precision=0.9851464778726258, recall=0.9836982404695586, fmeasure=0.984279538783495)),
 'rouge2': AggregateScore(low=Score(precision=0.9716482581189224, recall=0.9700250787394225, fmeasure=0.9707623506365148), mid=Score(precision=0.9730215106721225, recall=0.9715156641903945, fmeasure=0.9721647710199142), high=Score(precision=0.9743620065939519, recall=0.9729230313365992, fmeasure=0.9735553828918491)),
 'rougeL': AggregateScore(low=Score(precision=0.9824504386590022, recall=0.9808880918698047, fmeasure=0.981579063391171), mid=Score(precision=0.9834053482518403, recall=0.9818796565565557, fmeasure=0.9825250459119504), high=Score(precision=0.9843379333680138, recall=0.9827983958249998, fmeasure=0.9834433858242191)),
 'rougeLsum': AggregateScore(low=Score(precisi

In [ ]:
[{k: v.mid.fmeasure} for k,v in metrics.compute(predictions=predictions, references=references).items()]

[{'rouge1': 0.9834553737075293},
 {'rouge2': 0.9721647710199142},
 {'rougeL': 0.9825250459119504},
 {'rougeLsum': 0.9825770888165091}]

## Generate text

In [ ]:
sentence = input()
text =  sentence + " </s>"
encoding = tokenizer(text, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")
outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=300,
    early_stopping=True
)
for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(line)

Canh sat cho biet Kiptum la nguoi dieu khien chiec xe mat lai va lan banh khien ca hai tu vong tai cho Mot phat ngon vien duoc AFP dan loi noi them rang hanh khach thu ba la nu bi thuong va duoc dua den benh vien
Cảnh sát cho biết Kiptum là người điều khiển chiếc xe mất lái và lăn bánh khiến cả hai tử vong tại chỗ Một phát ngôn viên được AFP dẫn lời nói thêm rằng hành khách thứ ba là nữ bị thương và được đưa đến bệnh viện 
